# 餃子が食べたい！！

- あなたはいま餃子が食べたい
- 1000店舗の緯度経度情報・料理ジャンルを持っています。(CSVファイル)
- ある駅 (緯度 : 35.6721903 / 経度 :139.7363287) から1.5km以内にある餃子のお店を探す
- そのお店の【answer (回答項目)】が、問題1の回答です。

※ 緯度経度の座標はすべて世界測地系の座標

CSVファイル (1000店舗の緯度経度情報) はこちらからダウンロード
https://campaign-retty-me.s3.ap-northeast-1.amazonaws.com/prod/retty_gourmet_open/20210619/retty_gourmet_open_q1.csv


## 回答・解説
緯度経度とは、場所を数値的に表現する方法の一つ。

- 日本測地系 : 日本独自のもの。2002年に世界測地系へ移行している
- 世界測地系 : 国際的に定められた基準となる測地系が世界測地系

ご存知の通り、地球は平面ではなく球体 (正確には楕円体) になるので、球面を気にして計算する必要がある。

今回の回答例では、Python の GeoPy とよばれるジオコーディングライブラリを利用して実装。  
GeoPy では、カーニー法とよばれる、測地線距離を求める新しいアルゴリズムを採用。  
https://geopy.readthedocs.io/en/stable/#module-geopy.distance

## その他・備考
今回の問題に限っていくと、平面と捉えても正しい回答が導けます。  
三平方の定理を用いて、2点の距離を求めるで回答しても問題ありません。

In [9]:
import pandas as pd
from geopy.distance import geodesic
import math

In [10]:
df = pd.read_csv("retty_gourmet_open_q1.csv")
df.head()

,answer,latitude,longitude,category
0,PRE23/ARE64/SUB6401,35.170195,136.918005,餃子
1,PRE04/ARE264/SUB26401,38.186789,140.873993,和食
2,PRE40/ARE130/SUB13004,33.547256,130.336644,ステーキ
3,PRE26/ARE115/SUB11501,35.037039,135.730018,チャーハン
4,PRE23/ARE68/SUB6801,35.126077,136.879908,旅館


ある駅の緯度経度情報 (緯度 : 35.6697 / 経度 : 139.76714) と、店舗との距離を計算する

① GeoPyを利用した回答方法

In [18]:
station = (35.6721903, 139.7363287)

ans = []
dis = []
cat = []

for row in df.itertuples():
    ans.append(row[1])
    restaurant = (row[2], row[3])
    cat.append(row[4])
    t_dis = geodesic(station, restaurant).km 
    dis.append(t_dis)

df_dis = pd.DataFrame(index=[], columns=['answer', 'distance', 'category'])
df_dis['answer'] = ans
df_dis['distance'] = dis
df_dis['category'] = cat

print(df_dis.sort_values('distance').query('category=="餃子"').iloc[[0]])

                  answer  distance category
594  PRE13/ARE14/SUB1401  0.623166       餃子


② 平面と満たして、三平方の定理を利用して距離を出す方法

In [19]:
station = (35.6721903, 139.7363287)

ans = []
dis = []

def get_distance(x1, y1, x2, y2):
    d = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return d

for row in df.itertuples():
    ans.append(row[1])
    t_dis = get_distance(row[2], row[3], 35.6721903, 139.7363287)
    dis.append(t_dis)

df_dis = pd.DataFrame(index=[], columns=['answer', 'distance', 'category'])
df_dis['answer'] = ans
df_dis['distance'] = dis
df_dis['category'] = cat

print(df_dis.sort_values('distance').query('category=="餃子"').iloc[[0]])

                  answer  distance category
594  PRE13/ARE14/SUB1401   0.00681       餃子
